# Tangelo hands-on: Phase Estimation

## Before you jump in

Before tackling this hands-on, we suggest to have a look at:

- the tutorial on the basics of the linq module

You can use the cells below to install or import useful packages. We recommend to install qulacs as some of these circuits could be long.

In [3]:
import numpy as np
import time

If you wish to install anything in your jupyter environment while working on this notebook, including Tangelo, you can go through your terminal and reload the upyter kernel or run `pip` commands directly from this notebook using `!`

In [4]:
# Uncomment and change package name to install it in the current environment
# !pip install tangelo-gc --quiet
# !pip install qulacs --quiet

## Level 1: Exact statevector

Let's define a molecule in second quantization, for the sake of exploring the variational framework. Here we define a molecule explicitly, specifying a geometry, charge, spin and basis set, but you could also quickly just import other existing ones in our ["molecule library"](https://github.com/goodchemistryco/Tangelo/blob/main/tangelo/molecule_library.py) if you want to swap between other examples.

In [7]:
from openfermion import get_sparse_operator

from tangelo.toolboxes.operators import QubitOperator

# Generate qubit operator with state 9 having eigenvalue 0.25
qu_op = QubitOperator("X0 X1", 0.125) + QubitOperator("Y1 Y2", 0.125) + QubitOperator("Z2 Z3", 0.125) + QubitOperator("", 0.125)

ham_mat = get_sparse_operator(qu_op).toarray()
eigs, wavefunctions = np.linalg.eigh(ham_mat)

# Eigenvector 9 has eigenvalue 0.25
print(eigs[9])

0.25


In [14]:
from tangelo.linq import get_backend

# Choose backend with lsq_first (least significant qubit first) ordering
sim = get_backend("cirq")
print(f"sim has statevector ordering of {sim.backend_info()['statevector_order']}")

sim has statevector ordering of lsq_first


In [15]:
from tangelo.linq.helpers.circuits.statevector import StateVector

# Create the circuit that initialized the 9th
sv = StateVector(wavefunctions[:, 9], order=sim.backend_info()['statevector_order'])

sv_circuit = sv.initializing_circuit()


In [19]:
# check that state has correct energy
print(f"sv_circuit prepared a state with energy {sim.get_expectation_value(qubit_operator=qu_op, state_prep_circuit=sv_circuit)}")
n_state_qubits = sv_circuit.width
print(f"The state register is composed of {n_state_qubits}")

sv_circuit prepared a state with energy 0.25


Now you can prepare the circuit for phase estimation. Remember this figure.

![Phase Estimation Circuit](PE_Circuit.png "Phase Estimation Circuit")

In [ ]:
from tangelo.toolboxes.ansatz_generator.ansatz_utils import trotterize, get_qft_circuit
from tangelo.toolboxes.post_processing.histogram import Histogram
from tangelo.linq import Gate, Circuit

# Reverse order as cirq uses lsq_first
qubit_list = [6, 5, 4]

# State preparation
pe_circuit = sv_circuit + Circuit([Gate("H", q) for q in qubit_list ])

for i, qubit in enumerate(qubit_list):
    # You can play around with how accurate the time-evolution needs to be
    # Use negative time as trotterize uses exp(-iHt) to follow the Schrodinger equation
    pe_circuit += trotterize(qu_op, trotter_order=, n_trotter_steps=, time=, control=qubit)

# set inverse to true or false
pe_circuit += get_qft_circuit(qubit_list, inverse=True)

freqs, _ = sim.simulate(pe_circuit)

# Remove qubit indices from histogram corresponding to the state qubits i.e. (0, 1, 2, 3)
hist = Histogram(freqs)
hist.remove_qubit_indices(0, 1, 2, 3)

for key, probability in hist.frequencies.items():
      energy = sum(int(k)/2**(i+1) for i, k in enumerate(key))
      print(f"The probability that the statevector energy={energy} is {probability:3.5f}")

## Approximate example

In [93]:
# Define H2 molecule in minimal basis set
from tangelo import SecondQuantizedMolecule
from tangelo.toolboxes.qubit_mappings.mapping_transform import fermion_to_qubit_mapping
from tangelo.toolboxes.qubit_mappings.statevector_mapping import get_reference_circuit

xyz_H2 = [("H", (0., 0., 0.)), ("H", (0., 0., 0.7414))]
mol_H2_sto3g = SecondQuantizedMolecule(xyz_H2, q=0, spin=0, basis="sto-3g")

mol = mol_H2_sto3g

qu_op = fermion_to_qubit_mapping(mol.fermionic_hamiltonian, mapping="JW", n_spinorbitals=mol.n_active_sos, n_electrons=mol.n_active_electrons, spin=mol.spin, up_then_down=False)

# Cheat to get the ground state eigenvalue to be 0.125mod(1) exact ground state is 1.137270174660903
qu_op += (0.25 + 1.137270174660903)
eigs, statevectors = np.linalg.eigh(get_sparse_operator(qu_op).toarray())

print(eigs[0])
ground_sv = statevectors[:, 0]

0.24999999999999986


In [94]:
# Hartree-Fock reference state circuit
hf_circuit = get_reference_circuit(n_spinorbitals=mol.n_active_sos, n_electrons=mol.n_active_electrons, mapping="JW", up_then_down=False, spin=mol.spin)

f, sv = sim.simulate(hf_circuit, return_statevector=True)

print(f"The overlap of the initial state with the final state is {np.dot(sv, ground_sv)}")

The overlap of the initial state with the final state is (0.9936146058054713+0j)


Use phase-estimation to do state preparation. Warning, you will probably need very high order time-evolution to get a good result.

In [118]:
# Reverse order as cirq uses lsq_first
qubit_list = [6, 5, 4]

# State preparation
pe_circuit = hf_circuit + Circuit([Gate("H", q) for q in qubit_list ])

for i, qubit in enumerate(qubit_list):
    # You can play around with how accurate the time-evolution needs to be
    # Use negative time as trotterize uses exp(-iHt) to follow the Schrodinger equation
    pe_circuit += trotterize(qu_op, trotter_order=, n_trotter_steps=, time=, control=qubit)

# set inverse to true or false
pe_circuit += get_qft_circuit(qubit_list, inverse=)

sim = get_backend("cirq")
freqs, _ = sim.simulate(pe_circuit)

# Remove qubit indices from histogram corresponding to the state qubits i.e. (0, 1, 2, 3)
hist = Histogram(freqs)
hist.remove_qubit_indices(0, 1, 2, 3)

for key, probability in hist.frequencies.items():
      energy = sum(int(k)/2**(i+1) for i, k in enumerate(key))
      print(f"The probability that the statevector energy={energy} is {probability:3.5f}")

The probability that the statevector energy=0.0 is 0.00001
The probability that the statevector energy=0.125 is 0.00002
The probability that the statevector energy=0.25 is 0.99768
The probability that the statevector energy=0.375 is 0.00002
The probability that the statevector energy=0.5 is 0.00001
The probability that the statevector energy=0.625 is 0.00001
The probability that the statevector energy=0.75 is 0.00001
The probability that the statevector energy=0.875 is 0.00225


How good is our final state?

In [119]:
pe_plus_measure = pe_circuit + Circuit([Gate("MEASURE", i) for i in qubit_list])

_, sv_new = sim.simulate(pe_plus_measure, desired_meas_result="010", return_statevector=True)

# Shrink vector down to 2**4 size to compare with exact ground state.
sv_new_post = np.reshape(sv_new, (2**4, 2**3))[:, int("010", base=2)]

print(f"The final state overlap with the ground state is {np.abs(np.dot(sv_new_post, ground_sv))}")

The final state overlap with the ground state is 0.9978169790123522


Phase estimation is the backbone of many fault-tolerant quantum algorithms. Both as a state preparation method and a component of other algorithms such as HHL and Shor's algorithm.

What will you do with Tangelo ?